In [1]:
## Connect to DB server on AWS
import mysql.connector 
import config

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.pw
)
cursor = cnx.cursor()

In [42]:
# ## Create new DB 
# cursor.execute("CREATE DATABASE yelp")

In [130]:
# connect to database
cnx.database = 'yelp'

In [44]:
# # Create a table for the Businesses
# create_table_business = """
#     CREATE TABLE businesses (
#     businessID varchar(255) PRIMARY KEY,
#     name varchar(255),
#     rating FLOAT,
#     review_count INTEGER,
#     price varchar(255)
#     );

# """

In [45]:
# cursor.execute(create_table_business)

In [46]:
# create_table_review = """
#     CREATE TABLE reviews (
#     reviewID varchar(255) PRIMARY KEY,
#     text_of_review varchar(255),
#     businessID varchar(255),
#     FOREIGN KEY (businessID)
#         REFERENCES businesses(businessID)
#     );
# """

In [47]:
# cursor.execute(create_table_review)

In [2]:
import requests
import json

In [49]:
# Write a function to make a call to the yelp API
business_url = 'https://api.yelp.com/v3/businesses/search'

In [50]:
client_id = '2uBXLKdIeFK2UNqhwXOPRQ'
api_key = 'x_KJc7Xc556wdB3hXmC82fjHZ4hKR_KUzdaarXmcqnvkIi_7xJDme-Q4ze0TDTsY4BMag0PbgElDq2ax6Yn0u16KgnvWtVrrMEMjkRndEJZoDnURYccdtH4QNjTpXXYx'

In [51]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

In [52]:
# what type of business do you want to search
term = 'Spa'

#where do you want to perform this search
location = 'NYC'

# what is your other parameter you want to search against
status = True

In [53]:
url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'open_now' : status
            }


In [54]:
response = requests.get(business_url, headers=headers, params=url_params)

In [6]:
business_data = response.json()
business_data

NameError: name 'response' is not defined

# START HERE


###      

In [42]:
# ## Create new DB 
# cursor.execute("CREATE DATABASE yelp")

In [44]:
# # Create a table for the Businesses
# create_table_business = """
#     CREATE TABLE businesses (
#     businessID varchar(255) PRIMARY KEY,
#     name varchar(255),
#     rating FLOAT,
#     review_count INTEGER,
#     price varchar(255)
#     );

# """

In [45]:
# cursor.execute(create_table_business)

In [46]:
# create_table_review = """
#     CREATE TABLE reviews (
#     reviewID varchar(255) PRIMARY KEY,
#     text_of_review varchar(255),
#     businessID varchar(255),
#     FOREIGN KEY (businessID)
#         REFERENCES businesses(businessID)
#     );
# """

In [47]:
# cursor.execute(create_table_review)

In [3]:
import mysql.connector 
import config

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.pw
)
cursor = cnx.cursor()
cnx.database = 'yelp'

In [24]:
import requests
import json

business_url = 'https://api.yelp.com/v3/businesses/search'
client_id = '2uBXLKdIeFK2UNqhwXOPRQ'
api_key = 'x_KJc7Xc556wdB3hXmC82fjHZ4hKR_KUzdaarXmcqnvkIi_7xJDme-Q4ze0TDTsY4BMag0PbgElDq2ax6Yn0u16KgnvWtVrrMEMjkRndEJZoDnURYccdtH4QNjTpXXYx'


headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }


# what type of business do you want to search
term = 'Spa'

#where do you want to perform this search
location = 'NYC'

# what is your other parameter you want to search against
status = True


url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'open_now' : status
            }


def yelp_call(url_params, api_key):
    response = requests.get(business_url, headers=headers, params=url_params)
    business_data = response.json()
    return business_data

In [32]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

def parse_data(data):
    
    # create a container to hold our parsed data
    
    parsed_data = []
    
    # loop through our business and parse each individual business
    # in the loop, add each individual parsed business to our data container
    # return the container with all of the parsed results
    
    for business in data['businesses']:
        if 'price' not in business.keys():
            business.setdefault('price', None)
        biz_tuple = (business['id'], business['name'], business['rating'], business['review_count'], business['price'])
        parsed_data.append(biz_tuple)
    return parsed_data

NameError: name 'business_data' is not defined

In [30]:
# Write a function to take your parsed data and insert it into the DB

insert_business = """INSERT INTO businesses (businessID, name, rating, review_count, price) VALUES (%s, %s, %s, %s, %s);"""

In [31]:
cursor.executemany(insert_business, parsed_data)
cursor.fetchall()

NameError: name 'parsed_data' is not defined

In [129]:
cnx.commit()

In [2]:
cursor.execute("""DELETE FROM businesses;""")
cnx.commit()

In [22]:
def insert_data(cnx, cursor, parsed_data):
    insert_business = """INSERT INTO businesses (businessID, name, rating, review_count, price) VALUES (%s, %s, %s, %s, %s);"""
    cursor.executemany(insert_business, parsed_data)
    cnx.commit()
    return

In [23]:
# Write a script that combines the three functions above into a single process.

# create a variable  to keep track of which result you are in. 

cur = 0

#set up a while loop to go through and grab the result 
while cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    business_data = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_data = parse_data(business_data)
    
    # use your function to insert your parsed results into the db
    insert_data(cnx, cursor, parsed_data)
    #increment the counter by 50 to move on to the next results
    cur += 20

In [48]:
cursor.execute("""ALTER TABLE reviews
ADD date_created datetime;""")
cnx.commit()

In [33]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

cursor.execute("""SELECT businessID
                  FROM businesses;
                  """)
businessIDs = cursor.fetchall()
businessIDs

[('-2qK6QUAuany-IGh20_dZA',),
 ('-2wD7J_5qppBf6HCn3q-mw',),
 ('-7iEvCLrJUwpWInQR5sxlw',),
 ('-9L1IaG5wLWA9x6ZEfrJkg',),
 ('-BHqYAT5L4WgRCanrqFJpg',),
 ('-cWw5u1x7x4MKxZtxUDRDg',),
 ('-fIjJisGI-WC_Rm_IuRANA',),
 ('-i8kN8EJ_5IW8qd4ANGWYA',),
 ('-iPOQ-4bihfOjos4E37Gbg',),
 ('-kaktEKXsai7iNe0ba1FYw',),
 ('-LpRBbnl8S-qnX7keuBW-Q',),
 ('-R4lcVkMi05sQc2vQOTUiA',),
 ('-WIH0M47ImR3WuLNuhKEow',),
 ('-ylEp5z6uMJFbGtDmmMKJQ',),
 ('-z0ZupqnQVOf5M0mvFW3QA',),
 ('0bCGxIsPlpk1_Yv1OOX6zQ',),
 ('0crKQMROK2cNfHTcDOSQ6g',),
 ('0D5XW0YMA5ogUMHSp3cLSQ',),
 ('0gbyQuU4PXKRYPEcdbv6pw',),
 ('0Gsx9jxtGTuN6QOClevCvQ',),
 ('0mWhQkjlev_ZkqMIzKrvjw',),
 ('0NJ7JNesYLIOimhRKHR3fw',),
 ('0QhgfjdBu1RKyIrwygCMqg',),
 ('0r6vE07BV_ivZ5UfMvkXqQ',),
 ('0sKA_7c8TwDtvm63_bYTgw',),
 ('0TGmT4bd9cXYRCuG4HhwNg',),
 ('0tHNZVwHX_kIU5aeDaJzsg',),
 ('0wteTsW6iGG6QBIpKIOqlA',),
 ('0xEyTF5Te_MQJ9dYb9nPgA',),
 ('107bSHL9NlwxpTh7yXSI8w',),
 ('10EysmFMfoRh3DSI-0hnKA',),
 ('11MMADyWHFx4Ybf19bJmsA',),
 ('15gXug_kuObRUrKaGB0xBA',),
 ('1acqdCO

In [57]:
# write a function that takes a businessids 
# and makes a url for its review api


import requests
import json

client_id = '2uBXLKdIeFK2UNqhwXOPRQ'
api_key = 'x_KJc7Xc556wdB3hXmC82fjHZ4hKR_KUzdaarXmcqnvkIi_7xJDme-Q4ze0TDTsY4BMag0PbgElDq2ax6Yn0u16KgnvWtVrrMEMjkRndEJZoDnURYccdtH4QNjTpXXYx'


headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }


def yelp_review_urls(data):
    businessid_review_urls = []
    
    for businessid in data:
        review_url = "https://api.yelp.com/v3/businesses/" + businessid[0] + "/reviews"
        businessid_review_urls.append(review_url)
    return businessid_review_urls

In [58]:
review_urls = yelp_review_urls(businessIDs)
review_urls

# call review for one business only 
# 3 reviews
# insert
# go back to call review

['https://api.yelp.com/v3/businesses/-2qK6QUAuany-IGh20_dZA/reviews',
 'https://api.yelp.com/v3/businesses/-2wD7J_5qppBf6HCn3q-mw/reviews',
 'https://api.yelp.com/v3/businesses/-7iEvCLrJUwpWInQR5sxlw/reviews',
 'https://api.yelp.com/v3/businesses/-9L1IaG5wLWA9x6ZEfrJkg/reviews',
 'https://api.yelp.com/v3/businesses/-BHqYAT5L4WgRCanrqFJpg/reviews',
 'https://api.yelp.com/v3/businesses/-cWw5u1x7x4MKxZtxUDRDg/reviews',
 'https://api.yelp.com/v3/businesses/-fIjJisGI-WC_Rm_IuRANA/reviews',
 'https://api.yelp.com/v3/businesses/-i8kN8EJ_5IW8qd4ANGWYA/reviews',
 'https://api.yelp.com/v3/businesses/-iPOQ-4bihfOjos4E37Gbg/reviews',
 'https://api.yelp.com/v3/businesses/-kaktEKXsai7iNe0ba1FYw/reviews',
 'https://api.yelp.com/v3/businesses/-LpRBbnl8S-qnX7keuBW-Q/reviews',
 'https://api.yelp.com/v3/businesses/-R4lcVkMi05sQc2vQOTUiA/reviews',
 'https://api.yelp.com/v3/businesses/-WIH0M47ImR3WuLNuhKEow/reviews',
 'https://api.yelp.com/v3/businesses/-ylEp5z6uMJFbGtDmmMKJQ/reviews',
 'https://api.yelp.c

In [59]:
# write a function that takes a business id 
# and makes a call to the API for reivews

def yelp_review_call(url):
    response2 = requests.get(url, headers=headers)
    review_data = response2.json()
    return review_data

In [60]:
yelp_review_call(review_urls[0])

{'reviews': [{'id': 'CEx9xRKaK6zwAJ3_zpdntQ',
   'url': 'https://www.yelp.com/biz/spa-j-northvale?adjust_creative=2uBXLKdIeFK2UNqhwXOPRQ&hrid=CEx9xRKaK6zwAJ3_zpdntQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2uBXLKdIeFK2UNqhwXOPRQ',
   'text': 'My first experience here was over 5 years ago.  I received a gift card while I was attending a college nearby and had an excellent facial by Julie.\n\nI then...',
   'rating': 5,
   'time_created': '2016-04-01 16:29:40',
   'user': {'id': 'iFHQDWSLd6AsR1L-wKwDhg',
    'profile_url': 'https://www.yelp.com/user_details?userid=iFHQDWSLd6AsR1L-wKwDhg',
    'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/dDMKfSvVWVN_QJ7d1aKH8w/o.jpg',
    'name': 'Kozeta C.'}},
  {'id': 'KqLtYz1WxH8M_0Zngs9MDA',
   'url': 'https://www.yelp.com/biz/spa-j-northvale?adjust_creative=2uBXLKdIeFK2UNqhwXOPRQ&hrid=KqLtYz1WxH8M_0Zngs9MDA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2uBXLKdIeFK2UNqhwXOPRQ',
   'text': 

In [61]:
# Write a function to parse out the relevant information from the reviews

def parse_review_data(review_data):
    
    # create a container to hold our parsed data
    
    parsed_review_data = []
    
    # loop through our reviews and parse each individual review
    # in the loop, add each individual parsed review to our data container
    # return the container with all of the parsed results
    
    for review in review_data['reviews']:
        rev_tuple = (review['id'], review['text'], review['time_created'])
        parsed_review_data.append(rev_tuple)
    return parsed_review_data

In [66]:
# write a function to insert the parsed data into the reviews table

def insert_review_data(cnx, cursor, parsed_review_data):
    insert_review = """INSERT INTO reviews (reviewID, text_of_review, date_created) VALUES (%s, %s, %s);"""
    cursor.executemany(insert_review, parsed_review_data)
    cnx.commit()
    return

In [67]:
# combine the functions above into a single script

for url in review_urls:
    

# review_urls #all the urls for review API


#     def yelp_review_call(url):
#         response2 = requests.get(url, headers=headers)
#         review_data = response2.json()
#         return review_data
    
    review_data = yelp_review_call(url)


#     def parse_review_data(review_data):

#         # create a container to hold our parsed data

#         parsed_review_data = []

#         # loop through our reviews and parse each individual review
#         # in the loop, add each individual parsed review to our data container
#         # return the container with all of the parsed results

#         for review in review_data['reviews']:
#             rev_tuple = (review['id'], review['text'])
#             parsed_review_data.append(rev_tuple)
#         return parsed_review_data

    parsed_review_data = parse_review_data(review_data)


#     def insert_review_data(cnx, cursor, parsed_review_data):
#         insert_review = """INSERT INTO reviews (reviewID, text_of_review) VALUES (%s, %s);"""
#         cursor.executemany(insert_review, parsed_review_data)
#         cnx.commit()
#         return
    
    insert_review_data(cnx, cursor, parsed_review_data)

KeyError: 'reviews'

In [68]:
# cursor.execute("""DELETE FROM reviews""")
# cnx.commit()